In [3]:
import pandas as pd

In [99]:
data = pd.read_csv('Seasons_Stats.csv').fillna(value = 0)

In [124]:
data.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
5239,5727,1980.0,Kareem Abdul-Jabbar*,C,32.0,LAL,82.0,0.0,3143.0,25.3,...,0.765,190.0,696.0,886.0,371.0,81.0,280.0,297.0,216.0,2034.0
5240,5728,1980.0,Tom Abernethy,PF,25.0,GSW,67.0,0.0,1222.0,11.0,...,0.683,62.0,129.0,191.0,87.0,35.0,12.0,39.0,118.0,362.0
5241,5729,1980.0,Alvan Adams,C,25.0,PHO,75.0,0.0,2168.0,19.2,...,0.797,158.0,451.0,609.0,322.0,108.0,55.0,218.0,237.0,1118.0
5242,5730,1980.0,Tiny Archibald*,PG,31.0,BOS,80.0,80.0,2864.0,15.3,...,0.830,59.0,138.0,197.0,671.0,106.0,10.0,242.0,218.0,1131.0
5243,5731,1980.0,Dennis Awtrey,C,31.0,CHI,26.0,0.0,560.0,7.4,...,0.640,29.0,86.0,115.0,40.0,12.0,15.0,27.0,66.0,86.0


In [101]:
filter = data["Year"] >= 1980
data = data[filter]

In [102]:
len(data)

18927

In [17]:
mvps = {2017:'Russell Westbrook', 2016:'Stephen Curry', 2015:'Stephen Curry', 2014:'Kevin Durant', 2013:'LeBron James',2012:'LeBron James',2011:'Derrick Rose',2010:'LeBron James',2009:'LeBron James',2008:'Kobe Bryant',2007:'Dirk Nowitzki',2006:'Steve Nash',2005:'Steve Nash',2004:'Kevin Garnett',2003:'Tim Duncan',2002:'Tim Duncan',2001:'Allen Iveson',2000:'''Shaquille O'Neal''',1999:'Karl Malone',1998:'Michael Jordan',1997:'Karl Malone',1996:'Michael Jordan',1995:'David Robinson',1994:'Hakeem Olajuwon',1993:'Charles Barkley',1992:'Michael Jordan',1991:'Michael Jordan',1990:'Magic Johnson',1989:'Magic Johnson',1988:'Michael Jordan',1987:'Magic Johnson',1986:'Larry Bird',1985:'Larry Bird',1984:'Larry Bird',1983:'Moses Malone',1982:'Moses Malone',1981:'Julius Erving',1980:'Kareem Abdul-Jabbar'}

In [18]:
from collections import namedtuple

In [78]:
Player = namedtuple('Player', ['Year','Name', 'Pos', 'Games','MP','PER','TSr','ThreePAr','FTRate','ORBr','DRBr','TRBr','ASTr','STLr','BLKr','TOVr','USGr','OWS','DWS','WS','WSper48','OBPM','DBPM','BPM','VORP','FG','FGA','FGr','ThreeP','ThreePA','ThreePr','TwoP','TwoPA','TwoPr','eFGr','FT','FTA','FTr','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS'])

In [122]:
#Since row.FT%, row.3p, etc is not valid because a valid string cannot have % in it or start with a number, and I don't know how to escape it in this case, I have to use their indexes as below.
#11:TS%, 12:3PAr,14:ORB%,15:DRB%,16:TRB%,17:AST%,18:STL%,19:BLK%,20:TOV%,21:USG%,26:WS/48,34:FG%,35:3P,36:3PA,37:3P%,38:2P,39:2PA,40:2P%,41:eFG%,44:FT%

In [104]:
#Combine consecutive rows with the same player. Use sum of some values and average of other values.
data_refined = []
y = []
prev = None
count = 0
for row in data.itertuples():
    #When it's a different player from last row, divide existing data by count and add to result
    if prev != None and row.Player != prev.Name:
        tmp = Player(prev.Year,prev.Name,prev.Pos,prev.Games,prev.MP,prev.PER/count,prev.TSr/count,prev.ThreePAr/count,prev.FTRate/count,prev.ORBr/count,prev.DRBr/count,prev.TRBr/count,prev.ASTr/count,prev.STLr/count,prev.BLKr/count,prev.TOVr/count,prev.USGr/count,prev.OWS/count,prev.DWS/count,prev.WS/count,prev.WSper48/count,prev.OBPM/count,prev.DBPM/count,prev.BPM/count,prev.VORP/count,prev.FG,prev.FGA,prev.FGr/count,prev.ThreeP,prev.ThreePA,prev.ThreePr/count,prev.TwoP,prev.TwoPA,prev.TwoPr/count,prev.eFGr/count,prev.FT,prev.FTA,prev.FTr/count,prev.ORB,prev.DRB,prev.TRB,prev.AST,prev.STL,prev.BLK,prev.TOV,prev.PF,prev.PTS)
        data_refined.append(tmp)
        prev = Player(row.Year,row.Player,row.Pos,row.G,row.MP,row.PER,row[11],row[12],row.FTr,row[14],row[15],row[16],row[17],row[18],row[19],row[20],row[21],row.OWS,row.DWS,row.WS,row[26],row.OBPM,row.DBPM,row.BPM,row.VORP,row.FG,row.FGA,row[34],row[35],row[36],row[37],row[38],row[39],row[40],row[41],row.FT,row.FTA,row[44],row.ORB,row.DRB,row.TRB,row.AST,row.STL,row.BLK,row.TOV,row.PF,row.PTS)
        count = 1
    #When it's the same player as last row, update existing data
    elif prev != None and row.Player == prev.Name:
        prev = Player(row.Year,row.Player,row.Pos,prev.Games+row.G,prev.MP+row.MP,prev.PER+row.PER,prev.TSr+row[11],prev.ThreePAr+row[12],prev.FTRate+row.FTr,prev.ORBr+row[14],prev.DRBr+row[15],prev.TRBr+row[16],prev.ASTr+row[17],prev.STLr+row[18],prev.BLKr+row[19],prev.TOVr+row[20],prev.USGr+row[21],prev.OWS+row.OWS,prev.DWS+row.DWS,prev.WS+row.WS,prev.WSper48+row[26],prev.OBPM+row.OBPM,prev.DBPM+row.DBPM,prev.BPM+row.BPM,prev.VORP+row.VORP,prev.FG+row.FG,prev.FGA+row.FGA,prev.FGr+row[34],prev.ThreeP+row[35],prev.ThreePA+row[36],prev.ThreePr+row[37],prev.TwoP+row[38],prev.TwoPA+row[39],prev.TwoPr+row[40],prev.eFGr+row[41],prev.FT+row.FT,prev.FTA+row.FTA,prev.FTr+row[44],prev.ORB+row.ORB,prev.DRB+row.DRB,prev.TRB+row.TRB,prev.AST+row.AST,prev.STL+row.STL,prev.BLK+row.BLK,prev.TOV+row.TOV,prev.PF+row.PF,prev.PTS+row.PTS)
        count += 1
    #When it's the first row
    elif prev == None:
        prev = Player(row.Year,row.Player,row.Pos,row.G,row.MP,row.PER,row[11],row[12],row.FTr,row[14],row[15],row[16],row[17],row[18],row[19],row[20],row[21],row.OWS,row.DWS,row.WS,row[26],row.OBPM,row.DBPM,row.BPM,row.VORP,row.FG,row.FGA,row[34],row[35],row[36],row[37],row[38],row[39],row[40],row[41],row.FT,row.FTA,row[44],row.ORB,row.DRB,row.TRB,row.AST,row.STL,row.BLK,row.TOV,row.PF,row.PTS)
        count = 1

In [105]:
len(data_refined)

15519

In [106]:
data_refined[0]

Player(Year=1980.0, Name='Kareem Abdul-Jabbar*', Pos='C', Games=82.0, MP=3143.0, PER=25.300000000000001, TSr=0.63900000000000001, ThreePAr=0.001, FTRate=0.34399999999999997, ORBr=7.2000000000000002, DRBr=22.199999999999999, TRBr=15.4, ASTr=16.5, STLr=1.2, BLKr=4.5999999999999996, TOVr=15.699999999999999, USGr=24.100000000000001, OWS=9.5, DWS=5.2999999999999998, WS=14.800000000000001, WSper48=0.22699999999999998, OBPM=4.0, DBPM=2.7000000000000002, BPM=6.7000000000000002, VORP=6.7999999999999998, FG=835.0, FGA=1383.0, FGr=0.60399999999999998, ThreeP=0.0, ThreePA=1.0, ThreePr=0.0, TwoP=835.0, TwoPA=1382.0, TwoPr=0.60399999999999998, eFGr=0.60399999999999998, FT=364.0, FTA=476.0, FTr=0.76500000000000001, ORB=190.0, DRB=696.0, TRB=886.0, AST=371.0, STL=81.0, BLK=280.0, TOV=297.0, PF=216.0, PTS=2034.0)

In [107]:
#Some names ends with "*", so get rid of that. Get X(features) data and y(labels).
X_data = []
y_data = []
for d in data_refined:
    name = d.Name
    if name[-1] == '*':
        name = name[0:-1]
    if mvps[int(d.Year)] == name:
        y_data.append(1)
    else:
        y_data.append(0)
    
    features = [d.Games,d.MP,d.PER,d.TSr,d.ThreePAr,d.FTRate,d.ORBr,d.DRBr,d.TRBr,d.ASTr,d.STLr,d.BLKr,d.TOVr,d.USGr,d.OWS,d.DWS,d.WS,d.WSper48,d.OBPM,d.DBPM,d.BPM,d.VORP,d.FG,d.FGA,d.FGr,d.ThreeP,d.ThreePA,d.ThreePr,d.TwoP,d.TwoPA,d.TwoPr,d.eFGr,d.FT,d.FTA,d.FTr,d.ORB,d.DRB,d.TRB,d.AST,d.STL,d.BLK,d.TOV,d.PF,d.PTS]
    X_data.append(features)   

In [108]:
#From 1980-2017, 37 years so 37 mvps
sum(y_data)

37

In [109]:
len(y_data)

15519

In [110]:
X_training = X_data[:len(X_data)*3//4]
y_training = y_data[:len(X_data)*3//4]
X_testing = X_data[len(X_data)*3//4:]
y_testing = y_data[len(X_data)*3//4:]

In [112]:
len(X_testing)

3880

In [96]:
from sklearn.ensemble import RandomForestClassifier

In [97]:
def evaluate(pred, labels):
    TP=0
    TN=0
    FP=0
    FN=0
    for (a,b) in zip(pred, labels):
        if (a==True and b==True):
            TP = TP+1
        elif (a==True and b==False):
            FP = FP+1
        elif (a==False and b==True):
            FN = FN+1
        elif (a==False and b==False):
            TN = TN+1
    TPR = TP/(TP+FN)
    TNR = TN/(TN+FP)
    BER = 1-(TPR+TNR)/2
    print ("There are " + str(TP) + " true positives")
    print (str(TN) + " true negatives")
    print (str(FP) + " false positives")
    print (str(FN) + " false negatives")
    print ('accuracy is ' + str((TP + TN)/len(pred)))
    print ("BER is " + str(BER))

In [121]:
clf_RFC = RandomForestClassifier(max_depth=2, random_state=0,class_weight = {0:1,1:90})
clf_RFC.fit(X_training, y_training)
pred_RFC = clf_RFC.predict(X_testing)
evaluate(pred_RFC, y_testing)

There are 7 true positives
3842 true negatives
30 false positives
1 false negatives
accuracy is 0.9920103092783505
BER is 0.06637396694214881
